# 5. 텐서플로우 다중 레이어 뉴럴 네트워크
### -First Contact with TensorFlow

In [1]:
### 기본설정
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

# 그래프 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# batch_size만큼의 랜덤한 데이터 추출    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    X_batch, y_batch = [],[]
    for batch_idx in range(batch_size): # np.array_split(A,B) :  A array를 B개로 쪼개라
        X_batch.append(X[rnd_idx[batch_idx]])
        y_batch.append(y[rnd_idx[batch_idx]])
    return np.array(X_batch), arrayToOnehot(np.array(y_batch),10)

# 숫자를 OneHot으로 바꾸기
def arrayToOnehot(array, length):
    size = len(array)
    onehot = np.zeros((size,length))
    onehot[np.arange(size), array] = 1
    return onehot

# 데이터를 이미지로
def ShowImage(array):
    with tf.Session() as sess:
        image = np.array(array, dtype = 'float')
        pixels = image.reshape((28,28))
        plt.imshow(pixels, cmap ='gray')

## 콘볼루션 뉴럴 네트워크 (CNN)

- 특징  <br>
    - 이미지 데이터를 받는 것으로 뉴럴 네트워크를 효율적으로 구함
    - 가중치 행렬 W와 바이어스 b를 히든레이어의 모든 뉴런 이 공유
       <br>(필요한 가중치 파라메터의 수가 감소 -> 효율)
<br><br>
- 주 목적  <br>
테두리 (edge), 선 (line), 색깔 등 이미지의 시각적 특징이나 성질을 감지


### CNN을 통한 mnist 손글씨 이미지 인식

In [25]:
# 데이터 불어오기
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X_valid, y_valid = shuffle_batch(X_valid, y_valid, 5000)

In [3]:
x = tf.placeholder("float", shape=[None, 28*28])
y_ = tf.placeholder("float", shape=[None, 10])

In [4]:
# 입력 데이터를 원래 이미지의 크기로 재구성
x_image = tf.reshape(x, [-1,28,28,1])

두번째와 세번째의 차원 : 넓이와 높이
마지막 차원 : 컬러 채널
![콘뉴네1](https://tensorflowkorea.files.wordpress.com/2016/05/image072.png?w=300&h=282)

#### CNN의 두가지 기본원리
- 필터 (filter)  
<br>
- 특성 맵(characteristic map)

입력 데이터가 1st 히든레이어 뉴런에 완전히 연결되어 있지 않음  
<br>
![콘뉴네2](https://tensorflowkorea.files.wordpress.com/2016/05/image074.png?w=625)  
<br>
이 예에서 히든 레이어의 각 뉴런은 입력레이어의 5 * 5 영역과 연결  
<br>
이 5 * 5 영역의 윈도우가 28 * 28 의 입력 레이어를 쭉 훓는다고 생각하면 됨  
<br>
![콘뉴네3](https://tensorflowkorea.files.wordpress.com/2016/05/image076.png?w=625)  
<br>
즉, 위와 같은 그림이 됨

- 스트라이드(stride)  <br>
콘볼루션 레이어(Convoluted hiddon layer)에서 한번에 1픽셀 이상 움직일 수도 있음. 그때 사용하는 파라메터  
<br>
- 패딩(padding)  <br>
입력 이미지 밖으로 5 * 5 윈도우가 넘어갈 수 있도록 0(또는 다른 값)으로 테두리를 채우는 것

가중치 W와 바이어스 b를 CNN에서 보통 커널(kernel) 혹은 필터라 부름

하나의 커널은 이미지에서 한 종류의 특징만을 감지
<br>
따라서 감지하고 싶은 각 특징에 한개씩 여러 커널을 사용하는 것이 좋음
<br><br>
CNN에서는 완전한 콘볼루션 레이어는 여러게의 커널로 구성

![콘뉴네4](https://tensorflowkorea.files.wordpress.com/2016/05/image078.png?w=625)
<br><br>
이 예에서는 32개의 커널을 사용
<br><br>
각 커널은 5 * 5 가중치 행렬 W와 한개의 바이어스 b로 정의되고 이 히든 레이어의 뉴런들에 공통적으로 사용

In [7]:
# 가중치를 난수값(random noise)로 초기화
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# 바이어스를 작응 양수로 초기화
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

일반적으로 콘볼루션 레이어 외에 풀링(pooling) 레이어가 콘볼루션 레이어 뒤에 따라 옴
<br>
- 풀링 레이어
    - 콘볼루션 레이어의 출력을 단순화
    - 콘볼루션 레이어가 생산한 정보를 컴팩트한 버전으로 만듬
    
![콘뉴네5](https://tensorflowkorea.files.wordpress.com/2016/05/image080.png?w=625)
<br>
(이 예제에서는 풀링방법 중 max_pooling을 사용)
* max-pooling : 
2 * 2 영역에서 가장 큰 값을 선택해서 정보를 압축

![콘뉴네6](https://tensorflowkorea.files.wordpress.com/2016/05/image082.png?w=625)

### 모델 구현

In [10]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [16]:
# 콘볼루션 레이어1
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

[(윈도우 사이즈), (윈도우사이즈), (컬러 채널), (특징수)]

- ReLU (Rectified Linear Unit)
    - 딥 뉴럴 네트워크의 히든레이어에서 사용되는 기본 활성화 함수
    - 음수면 0을, 양수면 그대로 리턴

In [17]:
# 콘볼루션 레이어2
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

[(윈도우 사이즈), (윈도우사이즈), (이전 레이어의 출력값 크기), (특징수)]
<br>
12 * 12 크기의 행렬에 스트라이드 1로 5 * 5 윈도우를 적용했기 때문에 결과의 크기 7 * 7 

In [18]:
# 소프트맥스 레이어에 주입하기 위해 7 * 7 출력 값을 완전연결 레이어에 연결
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

[(콘볼루션레이어 크기 7 * &) * (필터 수), (임의로 선택한 뉴런 개수)]

In [19]:
# 텐서를 벡터로 변환 (소프트맥스에 넣기 위해)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

# 활성함수 적용
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

- 드롭 아웃 (dorpout)
    - 뉴럴 네트워크에서 필요한 파라메타 수를 줄임
    - 무작위로 노드를 삭제해 그 입력과 출력 연결을 제거
    - 오버피팅을 예방

In [20]:
# 드롭 아웃이 되지 않을 확률
keep_prob = tf.placeholder("float")

# 드롭 아웃
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [22]:
# 소프트 맥스 레이어
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

### 모델 훈련 및 평가

In [27]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        batch = shuffle_batch(X_train,y_train,50)
        if i%100 == 0:
            train_accuracy = sess.run( accuracy, feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
            print("step %d, training accuracy %g"%(i, train_accuracy))
        sess.run(train_step,feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print("test accuracy %g"% sess.run(accuracy, feed_dict={ x: X_valid, y_: y_valid, keep_prob: 1.0}))

step 0, training accuracy 0.1
test accuracy 0.8312
